In [ ]:
# ROR and PRR analysis using FAERS database
# Version v1.0
# Djamilla Simoens

# =============================
# 1. Install Dependencies
# =============================

# In Jupyter Notebook, run this cell first:
!pip install pandas numpy scipy openpyxl tqdm


In [ ]:
# =============================
# 2. Import The Needed libraries
# =============================

import pandas as pd
import numpy as np
import os
import glob
from scipy.stats import fisher_exact, chi2_contingency
from tqdm import tqdm


In [ ]:
# =============================
# 3. Input drug, reaction and timeline of interest
# =============================
# Type the name of the drug and adverse reactions of interest

drug_synonyms = ['alpelisib', 'PIQRAY']
event_synonyms = ['Stomatitis']
data_dir = './faers_data'  # Update with your folder with the FAERS data, make sure you have .txt files


# Define which quartely data you are interested in (e.g., from 2017Q3 to 2025Q1)
start_year_q = (2019, 2) # Year and number of quarter
end_year_q = (2025, 1) # Year and number of quarter

# Log skipped rows info
skipped_rows_log = []

def is_in_range(fname, start, end):
    import re
    match = re.search(r'(\d{2})(Q[1-4])', fname.upper())
    if not match:
        return False
    year_str, quarter_str = match.groups()
    year = int('20' + year_str) if int(year_str) < 50 else int('19' + year_str)
    quarter = int(quarter_str[1])
    return (year, quarter) >= start and (year, quarter) <= end


In [ ]:
# =============================
# 4. Load FAERS Data
# =============================

def load_files(folder, pattern, start_range, end_range):
    pattern_lower = pattern
    pattern_upper = pattern.upper()
    files = sorted(glob.glob(os.path.join(folder, pattern_lower)) +
                   glob.glob(os.path.join(folder, pattern_upper)))

    filtered_files = [f for f in files if is_in_range(os.path.basename(f), start_range, end_range)]

    dfs = []
    for file in tqdm(filtered_files, desc=f"Loading {pattern.split('*')[0]}"):
        try:
            df = pd.read_csv(
                file,
                sep='$',
                dtype=str,
                encoding='utf-8',
                on_bad_lines='skip',
                low_memory=False
            )
            dfs.append(df)
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(
                    file,
                    sep='$',
                    dtype=str,
                    encoding='latin1',
                    on_bad_lines='skip',
                    low_memory=False
                )
                dfs.append(df)
            except Exception as e:
                print(f"Failed to load {file}: {e}")
                skipped_rows_log.append((file, str(e)))
        except Exception as e:
            print(f"Failed to load {file}: {e}")
            skipped_rows_log.append((file, str(e)))

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

df_drug = load_files(data_dir, 'DRUG*.txt', start_year_q, end_year_q)
df_reac = load_files(data_dir, 'REAC*.txt', start_year_q, end_year_q)
print(f"Total DRUG records: {len(df_drug)}")
print(f"Total REACTION records: {len(df_reac)}")



In [ ]:
# =============================
# 5. Deduplication Following VigiMatch Standards
# =============================

print("Filtering suspect drugs...")
df_drug_ps = df_drug[df_drug['role_cod'].str.upper() == 'PS'].copy()
df_drug_ps['drugname'] = df_drug_ps['drugname'].fillna('').str.upper()
df_reac['pt'] = df_reac['pt'].fillna('').str.upper()

# Normalize and deduplicate drug names
def normalize_name(name):
    if not isinstance(name, str):
        return ''
    name = name.upper()
    for synonym in drug_synonyms:
        if synonym.upper() in name:
            return synonym.upper()
    return name

df_drug_ps['drugname_norm'] = df_drug_ps['drugname'].apply(normalize_name)

# Apply VigiMatch-style deduplication using following parameters: primaryid, drugname_norm, route, dose_vbm
dedup_columns = ['primaryid', 'drugname_norm', 'route', 'dose_vbm']
df_drug_ps_before = len(df_drug_ps)
df_drug_ps_unique = df_drug_ps.sort_values(by='primaryid').drop_duplicates(subset=dedup_columns)
df_drug_ps_after = len(df_drug_ps_unique)
duplicates_removed_drug = df_drug_ps_before - df_drug_ps_after
print(f" VigiMatch-style DRUG duplicates removed: {duplicates_removed_drug} (from {df_drug_ps_before} → {df_drug_ps_after})")


In [ ]:
# =============================
# 6. Matching Drug and Event
# =============================

isr_drug = set()
for name in drug_synonyms:
    name = name.upper()
    matches = df_drug_ps_unique[df_drug_ps_unique['drugname_norm'] == name]['primaryid'].unique()
    isr_drug.update(matches)

isr_event = set()
for name in event_synonyms:
    name = name.upper()
    matches = df_reac[df_reac['pt'].str.contains(name, na=False, regex=False)]['primaryid'].unique()
    isr_event.update(matches)


In [ ]:
# =============================
# 7. Create Variables for 2x2 Contingency Table
# =============================

A = len(isr_drug & isr_event)  # Drug + Event
B = len(isr_drug - isr_event)  # Drug + No Event
C = len(isr_event - isr_drug)  # Event + No Drug
all_isr = set(df_drug['primaryid'].unique())
D = len(all_isr - (isr_drug | isr_event))  # Neither

print(f"Contingency Table:\nA: {A}\nB: {B}\nC: {C}\nD: {D}")


In [ ]:
# =============================
# 8. Statistical Analysis
# =============================

# ROR calculation
contingency_table = np.array([[A, B], [C, D]])
ROR = (A / B) / (C / D) if B > 0 and C > 0 and D > 0 else np.nan

# PRR calculation
prr_numerator = A / (A + B) if (A + B) > 0 else np.nan
prr_denominator = (A + C) / (A + B + C + D) if (A + B + C + D) > 0 else np.nan
PRR = prr_numerator / prr_denominator if prr_denominator > 0 else np.nan

# Other calculations
oddsr, p_fisher = fisher_exact(contingency_table)
chi2, p_chi2, dof, expected = chi2_contingency(contingency_table)
SE = np.sqrt(1/A + 1/B + 1/C + 1/D) if A*B*C*D > 0 else np.nan
lower_CI = np.exp(np.log(ROR) - 1.96 * SE) if not np.isnan(SE) else np.nan
upper_CI = np.exp(np.log(ROR) + 1.96 * SE) if not np.isnan(SE) else np.nan




In [ ]:
# =============================
# 9. Results and Excel Data Output
# =============================

results = {
    'Database': 'FAERS',
    'Drug(s)': ", ".join(drug_synonyms),
    'Event(s)': ", ".join(event_synonyms),
    'Start Quarter': f"Q{start_year_q[1]} {start_year_q[0]}",
    'End Quarter': f"Q{end_year_q[1]} {end_year_q[0]}",
    'A (Drug+Event)': A,
    'B (Drug+No Event)': B,
    'C (Event+No Drug)': C,
    'D (Neither)': D,
    'ROR': ROR,
    'ROR 95% CI Lower': lower_CI,
    'ROR 95% CI Upper': upper_CI,
    'PRR': PRR,
    'Fisher p-value': p_fisher,
    'Chi2 p-value': p_chi2,
    'DRUG Duplicates Removed': duplicates_removed_drug,
    'Original DRUG Records': df_drug_ps_before,
    'Skipped Files': len(skipped_rows_log)
}

result_df = pd.DataFrame([results])
print(result_df)

output_file = 'FAERS_alpelisib_Term3_ROR_PRR_Results.xlsx'
result_df.to_excel(output_file, index=False)
print(f"Results exported to {output_file}")

if skipped_rows_log:
    pd.DataFrame(skipped_rows_log, columns=["File", "Error"]).to_csv("skipped_files_log.csv", index=False)
    print("Skipped file errors saved to 'skipped_files_log.csv'")
